In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

In [2]:
train_df = pd.read_csv(r"D:\다운로드\archive/newKDDTrain+.txt", header=None)

columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
    'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
    'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
    'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
    'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'class'
]
# Rename the columns of the DataFrame
train_df.columns = columns

train_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [3]:
test_df =  pd.read_csv(r"D:\다운로드\archive/newKDDTest+.txt", header=None)

test_columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
    'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
    'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
    'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
    'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'class'
]
# Rename the columns of the DataFrame
test_df.columns = columns

test_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,private,REJ,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly
1,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,anomaly
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,anomaly


In [4]:
kdd_df = pd.concat([train_df, test_df])

In [5]:
kdd_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [6]:
cicids_df = pd.read_csv("D:/dataset/cleaned_improved_cicids2017.csv")

In [7]:
kdd_df.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'class'],
      dtype='object')

In [8]:
cicids_df.columns

Index(['Protocol', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd RST Flags',
       'Bwd RST Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min',
       'Packet Length Max', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', '

In [9]:
kdd_df['service'].value_counts()

http         48191
private      26627
domain_u      9937
smtp          8247
ftp_data      7711
             ...  
tftp_u           4
http_8001        2
aol              2
harvest          2
http_2784        1
Name: service, Length: 70, dtype: int64

In [10]:
# 각 데이터셋의 컬럼 분포(값의 범위) 확인

def dataset_column_distribution(df, dataset_name):
    print(f"\n{dataset_name} Dataset Column Distributions:\n")
    for column in df.columns:
        col_data = df[column]
        if col_data.dtype in ['int64', 'float64']:
            print(f"{column}: Min={col_data.min()}, Max={col_data.max()}, Mean={col_data.mean():.2f}, Std={col_data.std():.2f}")
        else:
            print(f"{column}: Non-numeric column, Unique Values={col_data.nunique()}")

# KDD 데이터셋 컬럼 분포 확인
dataset_column_distribution(kdd_df, "KDD")

# CIC-IDS 데이터셋 컬럼 분포 확인
dataset_column_distribution(cicids_df, "CIC-IDS")


KDD Dataset Column Distributions:

duration: Min=0, Max=57715, Mean=276.78, Std=2460.68
protocol_type: Non-numeric column, Unique Values=3
service: Non-numeric column, Unique Values=70
flag: Non-numeric column, Unique Values=11
src_bytes: Min=0, Max=1379963888, Mean=40227.95, Std=5409611.60
dst_bytes: Min=0, Max=1309937401, Mean=17088.85, Std=3703524.78
land: Min=0, Max=1, Mean=0.00, Std=0.01
wrong_fragment: Min=0, Max=3, Mean=0.02, Std=0.24
urgent: Min=0, Max=3, Mean=0.00, Std=0.02
hot: Min=0, Max=101, Mean=0.19, Std=2.01
num_failed_logins: Min=0, Max=5, Mean=0.00, Std=0.07
logged_in: Min=0, Max=1, Mean=0.40, Std=0.49
num_compromised: Min=0, Max=7479, Mean=0.26, Std=22.23
root_shell: Min=0, Max=1, Mean=0.00, Std=0.04
su_attempted: Min=0, Max=2, Mean=0.00, Std=0.04
num_root: Min=0, Max=7468, Mean=0.27, Std=22.69
num_file_creations: Min=0, Max=100, Mean=0.01, Std=0.52
num_shells: Min=0, Max=5, Mean=0.00, Std=0.03
num_access_files: Min=0, Max=9, Mean=0.00, Std=0.10
num_outbound_cmds: Mi

In [11]:
kdd_df['protocol_type'].value_counts()

tcp     121569
udp      17614
icmp      9334
Name: protocol_type, dtype: int64

In [12]:
kdd_to_cicids_mapping = {
    'duration': 'Flow Duration',
    'protocol_type': 'Protocol',
    'src_bytes': 'Total Fwd Packet',
    'dst_bytes': 'Total Bwd packets',
    'count': 'Flow Packets/s',
    'srv_count': 'Flow Packets/s',
    'serror_rate': 'Fwd Packet Length Mean',
    'srv_serror_rate': 'Fwd Packet Length Std',
    'rerror_rate': 'Bwd Packet Length Mean',
    'srv_rerror_rate': 'Bwd Packet Length Std',
    'same_srv_rate': 'Fwd Packet Length Max',
    'diff_srv_rate': 'Fwd Packet Length Min',
    'dst_host_count': 'Fwd Packets/s',
    'dst_host_srv_count': 'Subflow Fwd Packets',
    'dst_host_same_srv_rate': 'Flow IAT Mean',
    'dst_host_diff_srv_rate': 'Flow IAT Std',
    'dst_host_serror_rate': 'Bwd IAT Mean',
    'dst_host_srv_serror_rate': 'Bwd IAT Std',
    'dst_host_rerror_rate': 'Packet Length Mean',
    'dst_host_srv_rerror_rate': 'Packet Length Std',
    'logged_in': 'Fwd PSH Flags',
    'hot': 'Fwd URG Flags',
    'srv_diff_host_rate': 'Bwd PSH Flags',
    'class': 'Label'
}

# KDD에서 공통 컬럼 추출 및 이름 변경
new_kdd_df = kdd_df[kdd_to_cicids_mapping.keys()].copy()
new_kdd_df.rename(columns=kdd_to_cicids_mapping, inplace=True)

# CIC-IDS에서 공통 컬럼 추출
cicids_columns = kdd_to_cicids_mapping.values()
new_cicids_df = cicids_df[list(cicids_columns)].copy()

# 결과 출력
print("Extracted KDD Dataset Columns:")
print(new_kdd_df.head())

print("Extracted CIC-IDS Dataset Columns:")
print(new_cicids_df.head())

Extracted KDD Dataset Columns:
   Flow Duration Protocol  Total Fwd Packet  Total Bwd packets  \
0              0      tcp               491                  0   
1              0      udp               146                  0   
2              0      tcp                 0                  0   
3              0      tcp               232               8153   
4              0      tcp               199                420   

   Flow Packets/s  Flow Packets/s  Fwd Packet Length Mean  \
0               2               2                     0.0   
1              13               1                     0.0   
2             123               6                     1.0   
3               5               5                     0.2   
4              30              32                     0.0   

   Fwd Packet Length Std  Bwd Packet Length Mean  Bwd Packet Length Std  ...  \
0                    0.0                     0.0                    0.0  ...   
1                    0.0                     

In [13]:
# 클리핑 적용
new_kdd_df['Fwd URG Flags'] = new_kdd_df['Fwd URG Flags'].clip(upper=10)
new_cicids_df['Fwd URG Flags'] = new_cicids_df['Fwd URG Flags'].clip(upper=10)

In [14]:
new_kdd_df

,Flow Duration,Protocol,Total Fwd Packet,Total Bwd packets,Flow Packets/s,Flow Packets/s,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Mean,Bwd Packet Length Std,...,Flow IAT Mean,Flow IAT Std,Bwd IAT Mean,Bwd IAT Std,Packet Length Mean,Packet Length Std,Fwd PSH Flags,Fwd URG Flags,Bwd PSH Flags,Label
0,0,tcp,491,0,2,2,0.0,0.0,0.0,0.0,...,0.17,0.03,0.00,0.00,0.05,0.00,0,0,0.00,normal
1,0,udp,146,0,13,1,0.0,0.0,0.0,0.0,...,0.00,0.60,0.00,0.00,0.00,0.00,0,0,0.00,normal
2,0,tcp,0,0,123,6,1.0,1.0,0.0,0.0,...,0.10,0.05,1.00,1.00,0.00,0.00,0,0,0.00,anomaly
3,0,tcp,232,8153,5,5,0.2,0.2,0.0,0.0,...,1.00,0.00,0.03,0.01,0.00,0.01,1,0,0.00,normal
4,0,tcp,199,420,30,32,0.0,0.0,0.0,0.0,...,1.00,0.00,0.00,0.00,0.00,0.00,1,0,0.09,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,794,333,1,1,0.0,0.0,0.0,0.0,...,0.72,0.06,0.01,0.00,0.00,0.00,1,0,0.00,normal
22540,0,tcp,317,938,2,11,0.0,0.0,0.0,0.0,...,1.00,0.00,0.01,0.00,0.00,0.00,1,0,0.18,normal
22541,0,tcp,54540,8314,5,10,0.0,0.0,0.0,0.0,...,1.00,0.00,0.00,0.00,0.07,0.07,1,2,0.20,anomaly
22542,0,udp,42,42,4,6,0.0,0.0,0.0,0.0,...,0.99,0.01,0.00,0.00,0.00,0.00,0,0,0.33,normal


In [15]:
new_cicids_df

,Flow Duration,Protocol,Total Fwd Packet,Total Bwd packets,Flow Packets/s,Flow Packets/s,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Mean,Bwd Packet Length Std,...,Flow IAT Mean,Flow IAT Std,Bwd IAT Mean,Bwd IAT Std,Packet Length Mean,Packet Length Std,Fwd PSH Flags,Fwd URG Flags,Bwd PSH Flags,Label
0,119719148,0,231,0,1.929516,1.929516,0.000000,0.000000,0.000000,0.000000,...,5.205180e+05,2.274526e+06,0.000000e+00,0.000000e+00,0.000000,0.000000,0,0,0,BENIGN
1,65511209,17,6,6,0.183175,0.183175,48.000000,0.000000,48.000000,0.000000,...,5.955564e+06,1.925808e+07,1.310222e+07,2.846299e+07,48.000000,0.000000,0,0,0,BENIGN
2,113976922,17,267,0,2.342579,2.342579,76.580524,44.140625,0.000000,0.000000,...,4.284847e+05,3.701805e+06,0.000000e+00,0.000000e+00,76.580524,44.140625,0,0,0,BENIGN
3,67037196,17,8,8,0.238673,0.238673,48.000000,0.000000,48.000000,0.000000,...,4.469146e+06,1.427166e+07,9.570156e+06,2.047819e+07,48.000000,0.000000,0,0,0,BENIGN
4,68045057,17,8,8,0.235138,0.235138,48.000000,0.000000,48.000000,0.000000,...,4.536337e+06,1.452505e+07,9.712504e+06,2.085587e+07,48.000000,0.000000,0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099966,64,6,1,1,31250.000000,31250.000000,0.000000,0.000000,0.000000,0.000000,...,6.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0,0,0,Portscan
2099967,76,6,1,1,26315.789474,26315.789474,0.000000,0.000000,0.000000,0.000000,...,7.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0,0,0,Portscan
2099968,81,6,1,1,24691.358025,24691.358025,0.000000,0.000000,0.000000,0.000000,...,8.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0,0,0,Portscan
2099969,297,17,2,2,13468.013468,13468.013468,34.000000,0.000000,139.000000,0.000000,...,9.900000e+01,1.319242e+02,4.700000e+01,0.000000e+00,86.500000,60.621778,0,0,0,BENIGN


In [16]:
new_cicids_df['Subflow Fwd Packets'].value_counts()

0    2044907
1      55064
Name: Subflow Fwd Packets, dtype: int64

In [17]:
# 이진분류
new_cicids_df['Label'] = new_cicids_df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)

# 변경된 데이터프레임 확인
print(new_cicids_df.head())

   Flow Duration  Protocol  Total Fwd Packet  Total Bwd packets  \
0      119719148         0               231                  0   
1       65511209        17                 6                  6   
2      113976922        17               267                  0   
3       67037196        17                 8                  8   
4       68045057        17                 8                  8   

   Flow Packets/s  Flow Packets/s  Fwd Packet Length Mean  \
0        1.929516        1.929516                0.000000   
1        0.183175        0.183175               48.000000   
2        2.342579        2.342579               76.580524   
3        0.238673        0.238673               48.000000   
4        0.235138        0.235138               48.000000   

   Fwd Packet Length Std  Bwd Packet Length Mean  Bwd Packet Length Std  ...  \
0               0.000000                     0.0                    0.0  ...   
1               0.000000                    48.0                    0.

In [18]:
new_kdd_df['Label'].value_counts()

normal     77054
anomaly    71463
Name: Label, dtype: int64

In [19]:
new_kdd_df['Label'] = new_kdd_df['Label'].apply(lambda x: 0 if x == 'normal' else 1)


In [20]:
new_kdd_df['Label'].value_counts()

0    77054
1    71463
Name: Label, dtype: int64

In [21]:
new_kdd_df.columns==new_cicids_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [22]:
print("cicids shape : ",new_cicids_df.shape)
print("kdd shape : ",new_kdd_df.shape)


cicids shape :  (2099971, 24)
kdd shape :  (148517, 24)


In [23]:
new_kdd_df

,Flow Duration,Protocol,Total Fwd Packet,Total Bwd packets,Flow Packets/s,Flow Packets/s,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Mean,Bwd Packet Length Std,...,Flow IAT Mean,Flow IAT Std,Bwd IAT Mean,Bwd IAT Std,Packet Length Mean,Packet Length Std,Fwd PSH Flags,Fwd URG Flags,Bwd PSH Flags,Label
0,0,tcp,491,0,2,2,0.0,0.0,0.0,0.0,...,0.17,0.03,0.00,0.00,0.05,0.00,0,0,0.00,0
1,0,udp,146,0,13,1,0.0,0.0,0.0,0.0,...,0.00,0.60,0.00,0.00,0.00,0.00,0,0,0.00,0
2,0,tcp,0,0,123,6,1.0,1.0,0.0,0.0,...,0.10,0.05,1.00,1.00,0.00,0.00,0,0,0.00,1
3,0,tcp,232,8153,5,5,0.2,0.2,0.0,0.0,...,1.00,0.00,0.03,0.01,0.00,0.01,1,0,0.00,0
4,0,tcp,199,420,30,32,0.0,0.0,0.0,0.0,...,1.00,0.00,0.00,0.00,0.00,0.00,1,0,0.09,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,794,333,1,1,0.0,0.0,0.0,0.0,...,0.72,0.06,0.01,0.00,0.00,0.00,1,0,0.00,0
22540,0,tcp,317,938,2,11,0.0,0.0,0.0,0.0,...,1.00,0.00,0.01,0.00,0.00,0.00,1,0,0.18,0
22541,0,tcp,54540,8314,5,10,0.0,0.0,0.0,0.0,...,1.00,0.00,0.00,0.00,0.07,0.07,1,2,0.20,1
22542,0,udp,42,42,4,6,0.0,0.0,0.0,0.0,...,0.99,0.01,0.00,0.00,0.00,0.00,0,0,0.33,0


In [24]:
# 프로토콜 매핑 사전
protocol_map = {
    'tcp': 6,    # TCP
    'udp': 17,   # UDP
    'icmp': 1    # ICMP
}

# 나머지 값은 0으로 처리
new_kdd_df['Protocol_Mapped'] = new_kdd_df['Protocol'].str.lower().map(protocol_map).fillna(0).astype(int)

new_kdd_df['Protocol'] = new_kdd_df['Protocol_Mapped']
new_kdd_df = new_kdd_df.drop(labels='Protocol_Mapped', axis=1)

In [25]:
new_kdd_df['Protocol'].value_counts()

6     121569
17     17614
1       9334
Name: Protocol, dtype: int64

In [26]:
new_kdd_df.shape

(148517, 24)

In [27]:
new_cicids_df.shape

(2099971, 24)

In [28]:
from sklearn.preprocessing import StandardScaler

# 데이터 정규화 함수
def apply_standard_scaling(df):
    scaler = StandardScaler()
    features = df.drop(columns=['Label'])  # 라벨 제외
    labels = df['Label']
    
    # StandardScaler 적용
    features_scaled = scaler.fit_transform(features)
    
    # 정규화된 데이터프레임 생성
    scaled_df = pd.DataFrame(features_scaled, columns=features.columns)
    scaled_df['Label'] = labels.reset_index(drop=True)
    
    return scaled_df

cicids_scaled_df = apply_standard_scaling(new_cicids_df)

kdd_scaled_df = apply_standard_scaling(new_kdd_df)


In [29]:
X_train_normal = cicids_scaled_df[cicids_scaled_df['Label'] == 0]


In [30]:
X_train_normal.shape

(1594540, 24)

In [31]:
X_test = kdd_scaled_df

In [32]:
y_train_normal = X_train_normal['Label']
X_train_normal= X_train_normal.drop(labels='Label', axis=1)

y_test = X_test['Label']
X_test= X_test.drop(labels='Label', axis=1)

In [33]:
X_train_normal

,Flow Duration,Protocol,Total Fwd Packet,Total Bwd packets,Flow Packets/s,Flow Packets/s,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Mean,Bwd Packet Length Std,...,Subflow Fwd Packets,Flow IAT Mean,Flow IAT Std,Bwd IAT Mean,Bwd IAT Std,Packet Length Mean,Packet Length Std,Fwd PSH Flags,Fwd URG Flags,Bwd PSH Flags
0,3.457206,-2.040642,0.251017,-0.012270,-0.180761,-0.180761,-0.542756,-0.391062,-0.611630,-0.486565,...,6.094006,-0.097796,0.093397,-0.213260,-0.271586,-0.722388,-0.572061,-0.165695,-0.006039,-0.069095
1,1.710282,1.049137,-0.007492,-0.007088,-0.180777,-0.180777,-0.019357,-0.391062,-0.540078,-0.486565,...,-0.164096,1.231036,2.896883,1.132794,6.019899,-0.558845,-0.572061,-0.165695,-0.006039,-0.069095
2,3.272155,1.049137,0.292378,-0.012270,-0.180757,-0.180757,0.292289,-0.106479,-0.611630,-0.486565,...,6.094006,-0.120298,0.328999,-0.213260,-0.271586,-0.461468,-0.505397,-0.165695,-0.006039,-0.069095
3,1.759459,1.049137,-0.005194,-0.005361,-0.180777,-0.180777,-0.019357,-0.391062,-0.540078,-0.486565,...,-0.164096,0.867617,2.073771,0.769928,4.254932,-0.558845,-0.572061,-0.165695,-0.006039,-0.069095
4,1.791939,1.049137,-0.005194,-0.005361,-0.180777,-0.180777,-0.019357,-0.391062,-0.540078,-0.486565,...,-0.164096,0.884044,2.115599,0.784552,4.338414,-0.558845,-0.572061,-0.165695,-0.006039,-0.069095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099958,-0.399335,1.049137,-0.012088,-0.010542,-0.180011,-0.180011,-0.172015,-0.391062,-0.435730,-0.486565,...,-0.164096,-0.221086,-0.277422,-0.213260,-0.271586,-0.463446,-0.498817,-0.165695,-0.006039,-0.069095
2099962,-0.397606,1.049137,-0.012088,-0.010542,-0.180413,-0.180413,-0.150207,-0.391062,-0.474488,-0.486565,...,-0.164096,-0.216715,-0.272304,-0.213260,-0.271586,-0.504331,-0.523232,-0.165695,-0.006039,-0.069095
2099965,-0.343384,1.049137,-0.012088,-0.010542,-0.180758,-0.180758,-0.193823,-0.391062,-0.467034,-0.486565,...,-0.164096,-0.079592,-0.111952,-0.213260,-0.271586,-0.502628,-0.515384,-0.165695,-0.006039,-0.069095
2099969,-0.400896,1.049137,-0.012088,-0.010542,-0.054748,-0.054748,-0.172015,-0.391062,-0.404426,-0.486565,...,-0.164096,-0.225035,-0.282039,-0.213255,-0.271586,-0.427671,-0.480506,-0.165695,-0.006039,-0.069095


In [34]:
X_test

,Flow Duration,Protocol,Total Fwd Packet,Total Bwd packets,Flow Packets/s,Flow Packets/s,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Mean,Bwd Packet Length Std,...,Subflow Fwd Packets,Flow IAT Mean,Flow IAT Std,Bwd IAT Mean,Bwd IAT Std,Packet Length Mean,Packet Length Std,Fwd PSH Flags,Fwd URG Flags,Bwd PSH Flags
0,-0.112481,-0.256213,-0.007346,-0.004614,-0.696611,-0.348310,-0.594848,-0.590270,-0.406460,-0.405192,...,-0.849240,-0.813554,-0.278736,-0.597720,-0.584813,-0.267151,-0.406815,-0.821249,-0.121524,-0.376389
1,-0.112481,2.589586,-0.007409,-0.004614,-0.602401,-0.361578,-0.594848,-0.590270,-0.406460,-0.405192,...,-1.065006,-1.192968,2.657874,-0.597720,-0.584813,-0.422074,-0.406815,-0.821249,-0.121524,-0.376389
2,-0.112481,-0.256213,-0.007436,-0.004614,0.339700,-0.295238,1.720415,1.721459,-0.406460,-0.405192,...,-0.840250,-0.969783,-0.175697,1.736012,1.742295,-0.422074,-0.406815,-0.821249,-0.121524,-0.376389
3,-0.112481,-0.256213,-0.007394,-0.002413,-0.670918,-0.308506,-0.131796,-0.127924,-0.406460,-0.405192,...,1.218511,1.038878,-0.433294,-0.527708,-0.561542,-0.422074,-0.376989,1.217658,-0.121524,-0.376389
4,-0.112481,-0.256213,-0.007400,-0.004501,-0.456804,0.049729,-0.594848,-0.590270,-0.406460,-0.405192,...,1.218511,1.038878,-0.433294,-0.597720,-0.584813,-0.422074,-0.406815,1.217658,-0.121524,-0.028744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,-0.112481,-0.256213,-0.007290,-0.004524,-0.705176,-0.361578,-0.594848,-0.590270,-0.406460,-0.405192,...,0.193625,0.413961,-0.124178,-0.574382,-0.584813,-0.422074,-0.406815,1.217658,-0.121524,-0.376389
148513,-0.112481,-0.256213,-0.007378,-0.004361,-0.696611,-0.228898,-0.594848,-0.590270,-0.406460,-0.405192,...,1.218511,1.038878,-0.433294,-0.574382,-0.584813,-0.422074,-0.406815,1.217658,-0.121524,0.318901
148514,-0.112481,-0.256213,0.002646,-0.002369,-0.670918,-0.242166,-0.594848,-0.590270,-0.406460,-0.405192,...,1.218511,1.038878,-0.433294,-0.597720,-0.584813,-0.205182,-0.198035,1.217658,2.301080,0.396156
148515,-0.112481,2.589586,-0.007429,-0.004603,-0.679482,-0.295238,-0.594848,-0.590270,-0.406460,-0.405192,...,1.191540,1.016560,-0.381775,-0.597720,-0.584813,-0.422074,-0.406815,-0.821249,-0.121524,0.898310


In [35]:
y_test

0         0
1         0
2         1
3         0
4         0
         ..
148512    0
148513    0
148514    1
148515    0
148516    1
Name: Label, Length: 148517, dtype: int64

In [36]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector, Conv1D, Conv1DTranspose
from keras.models import Model
from keras import regularizers
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.callbacks import EarlyStopping

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, Model, Input
import numpy as np
from tensorflow.keras import backend as K

latent_dim = 10
inter_dim = 20

# Sampling function for reparameterization trick
def sampling(args):
    z_mean, z_log_sigma = args
    batch_size = tf.shape(z_mean)[0]
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1.)
    return z_mean + z_log_sigma * epsilon

def vae_loss(x, x_decoded_mean, z_mean, z_log_sigma):
    reconstruction_loss = K.sum(K.square(x - x_decoded_mean), axis=1)
    kl_loss = -0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    kl_loss_weighted = kl_loss * 0.001  # KL 다이버전스 손실에 가중치 부여
    total_loss = K.mean(reconstruction_loss + kl_loss_weighted)
    return total_loss

def cvae(X, labels):
    features = X.shape[1]  # 2D 데이터이므로 timesteps는 필요 없고, features만 사용
    input_x = Input(shape=(features,), name='InputFeatures')  # 2D 입력 (샘플, 특징)

    input_label = Input(shape=(1,), name='InputLabel')  # Assumes binary or multi-class label

    # Embed the label into the same shape as the input
    embedded_label = layers.Embedding(input_dim=2, output_dim=features)(input_label)  # Adjust output_dim to match features
    embedded_label = layers.Flatten()(embedded_label)

    # Concatenate input_x and embedded_label
    concatenated_input = layers.Concatenate()([input_x, embedded_label])

    # Reshape for Conv1D layer (to 3D)
    reshaped_input = layers.Reshape((features + embedded_label.shape[-1], 1))(concatenated_input)

    # Encoder with CNN layers
    h = layers.Conv1D(filters=64, kernel_size=3, activation="relu", padding='same')(reshaped_input)
    h = layers.Conv1D(filters=32, kernel_size=3, activation="relu", padding='same')(h)
    h = layers.Conv1D(filters=16, kernel_size=3, activation="relu", padding='same')(h)

    # Flatten for dense layers (MLP)
    h = layers.Flatten()(h)
    h = layers.Dense(inter_dim, activation='relu')(h)
    h = layers.Dense(inter_dim, activation='relu')(h)

    # z layer
    z_mean = layers.Dense(latent_dim)(h)
    z_log_sigma = layers.Dense(latent_dim)(h)
    z = layers.Lambda(sampling)([z_mean, z_log_sigma])

    # Decoder
    # Add the label to the latent space z
    z_with_label = layers.Concatenate()([z, embedded_label])

    # Expand for Conv1DTranspose layer
    decoder1 = layers.Dense((features + embedded_label.shape[-1]) * inter_dim)(z_with_label)
    decoder1 = layers.Reshape((features + embedded_label.shape[-1], inter_dim))(decoder1)

    # Decoder with CNN layers
    decoder1 = layers.Conv1DTranspose(filters=16, kernel_size=3, activation="relu", padding='same')(decoder1)
    decoder1 = layers.Conv1DTranspose(filters=32, kernel_size=3, activation="relu", padding='same')(decoder1)
    decoder1 = layers.Conv1DTranspose(filters=64, kernel_size=3, activation="relu", padding='same')(decoder1)

    # Flatten and final dense layer to reconstruct original features
    decoder1 = layers.Flatten()(decoder1)
    decoder1 = layers.Dense(features)(decoder1)
    
    model = Model([input_x, input_label], decoder1)
    model.add_loss(vae_loss(input_x, decoder1, z_mean, z_log_sigma))
    
    return model

# Create CVAE model
model = cvae(X_train_normal, y_train_normal)
model.summary()

from tensorflow.keras.callbacks import EarlyStopping
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy'])
# Create an EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

# Compile and train the model with early stopping
history = model.fit([X_train_normal, y_train_normal], X_train_normal,
                    shuffle=True,
                    epochs=50, 
                    validation_split=0.1,  
                    batch_size=32,
                    callbacks=[early_stopping]).history

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 InputLabel (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 23)        46          ['InputLabel[0][0]']             
                                                                                                  
 InputFeatures (InputLayer)     [(None, 23)]         0           []                               
                                                                                                  
 flatten (Flatten)              (None, 23)           0           ['embedding[0][0]']              
                                                                                              

                                                                                                  
 tf.__operators__.add_1 (TFOpLa  (None,)             0           ['tf.math.reduce_sum[0][0]',     
 mbda)                                                            'tf.math.multiply_1[0][0]']     
                                                                                                  
 tf.math.reduce_mean (TFOpLambd  ()                  0           ['tf.__operators__.add_1[0][0]'] 
 a)                                                                                               
                                                                                                  
 add_loss (AddLoss)             ()                   0           ['tf.math.reduce_mean[0][0]']    
                                                                                                  
Total params: 131,377
Trainable params: 131,377
Non-trainable params: 0
_____________________________________

In [ ]:
# plot the training losses
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

In [ ]:
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

In [ ]:
valid_x_predictions = model.predict([X_test, y_test])
#mse = np.mean(np.power(flatten(test_X_selected) - flatten(valid_x_predictions), 2), axis=1)

mse = np.mean(np.power(X_test - valid_x_predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_test.tolist()})

In [ ]:
error_df

In [ ]:
error_df[error_df['true_class']==0]

In [ ]:
import matplotlib.pyplot as plt

# 데이터프레임 준비
true_class = error_df['true_class'].astype(str)
reconstruction_error = error_df['reconstruction_error']

# 박스 플롯 그리기
plt.figure(figsize=(8, 8))
plt.boxplot(
    [reconstruction_error[true_class == cls] for cls in sorted(true_class.unique())],
    labels=sorted(true_class.unique()),
    showfliers=False,
    vert=True,
    patch_artist=True
)

plt.ylabel('Distribution')
plt.xticks(rotation=90)
plt.show()


In [ ]:


import numpy as np
from sklearn.metrics import f1_score

# Define the range of threshold values
threshold_range = np.arange(0, 4, 0.1)

# List to store F1 scores for each threshold
f1_scores = []

# Loop through each threshold and calculate F1 score
for threshold in threshold_range:
    y_pred = [0 if e < threshold else 1 for e in error_df.reconstruction_error.values]
    f1 = f1_score(error_df.true_class, y_pred)
    f1_scores.append(f1)

# Find the threshold with the highest F1 score
best_threshold = threshold_range[np.argmax(f1_scores)]
best_f1_score = max(f1_scores)

print(f"Best threshold: {best_threshold}")
print(f"Best F1 score: {best_f1_score}")

# Optionally, you can plot the F1 scores across the threshold range
plt.plot(threshold_range, f1_scores, marker='o')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Score vs Threshold')
plt.show()


In [ ]:
# 그래프 보고 threshold 결정 0.35
threshold = best_threshold

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Prepare the data for plotting
true_class_0 = error_df[error_df['true_class'] == 0]['reconstruction_error']
true_class_1 = error_df[error_df['true_class'] == 1]['reconstruction_error']

# Create the figure and axes with a specified y-axis limit
plt.figure(figsize=(8, 6))

# Add jitter to avoid overlapping points in the scatter plot
x_0 = np.random.normal(1, 0.04, size=len(true_class_0))  # Jitter for class 0
x_1 = np.random.normal(2, 0.04, size=len(true_class_1))  # Jitter for class 1

# Create the scatter plot
plt.scatter(x_0, true_class_0, color='orange', alpha=0.6, edgecolor='black', label='Class 0')
plt.scatter(x_1, true_class_1, color='blue', alpha=0.6, edgecolor='black', label='Class 1')

# Set y-axis limit
plt.ylim(0, 0.5)

# Set x-axis labels and adjust ticks
plt.xticks([1, 2], ['0', '1'])
plt.xlabel('True Class')

# Set y-axis label and title
plt.ylabel('Reconstruction Error')
plt.title('Reconstruction Error by True Class')

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Prepare the data for plotting
true_class_0 = error_df[error_df['true_class'] == 0]['reconstruction_error']
true_class_1 = error_df[error_df['true_class'] == 1]['reconstruction_error']

# Create the figure and axes
fig, ax = plt.subplots(figsize=(8, 8))

# Create a swarm plot equivalent using scatter plot with more jitter
x_0 = np.random.normal(1, 0.1, size=len(true_class_0))  # Increased jitter for class 0
x_1 = np.random.normal(2, 0.1, size=len(true_class_1))  # Increased jitter for class 1

ax.scatter(x_0, true_class_0, color='blue', alpha=0.6, label='Class 0', edgecolor='w', s=50)
ax.scatter(x_1, true_class_1, color='orange', alpha=0.6, label='Class 1', edgecolor='w', s=50)

# Create boxplots
ax.boxplot([true_class_0, true_class_1], positions=[1, 2], widths=0.6, patch_artist=True, 
           showfliers=False, boxprops=dict(facecolor='None', color='black'),
           medianprops=dict(color='black'), whiskerprops=dict(color='black'))

# Set x-axis labels
ax.set_xticks([1, 2])
ax.set_xticklabels(['Class 0', 'Class 1'])

# Add a horizontal line at the threshold
ax.axhline(y=0.03, xmin=0.0, xmax=1, dashes=(5, 5), color='red')

# Adjust y-axis limit to make sure the threshold line is visible
#ax.set_ylim(bottom=min(min(true_class_0), min(true_class_1)) - 0.01, 
#           top=max(max(true_class_0), max(true_class_1)) + 0.01)
ax.set_ylim(bottom=0, top=1)
    
# Set labels and title
ax.set_ylabel('Reconstruction Error')
ax.set_title('Reconstruction Error by Class')

plt.show()


In [ ]:
groups = error_df.groupby('true_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, group.reconstruction_error, marker='^', ms=3.5, linestyle='',
            label= "Normal" if name == 0 else "Fall")
ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.set_ylim(0, 4)
ax.legend()
plt.title("Reconstruction error for different classes")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Assuming you have already defined LABELS, threshold, and y_pred

LABELS = ["Attack", "Normal"]

y_pred = [0 if e < threshold else 1 for e in error_df.reconstruction_error.values]
conf_matrix = confusion_matrix(error_df.true_class, y_pred)

# Create the figure and axis
plt.figure(figsize=(12, 12))
fig, ax = plt.subplots()

# Plot the confusion matrix using imshow
cax = ax.matshow(conf_matrix, cmap=plt.cm.Blues)

# Add color bar
plt.colorbar(cax)

# Set the labels for the axes
ax.set_xticks(np.arange(len(LABELS)))
ax.set_yticks(np.arange(len(LABELS)))

ax.set_xticklabels(LABELS)
ax.set_yticklabels(LABELS)

# Set labels and title
plt.xlabel('Predicted class')
plt.ylabel('True class')
plt.title('Confusion matrix')

# Annotate the confusion matrix with the counts
for i in range(len(LABELS)):
    for j in range(len(LABELS)):
        ax.text(j, i, format(conf_matrix[i, j], 'd'),
                ha="center", va="center",
                color="white" if conf_matrix[i, j] > conf_matrix.max() / 2 else "black")

plt.show()


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

precision, recall, f1,_ = precision_recall_fscore_support(y_test,y_pred,average='binary')
print ('Accuracy Score :',accuracy_score(error_df.true_class, y_pred) )
print ('Precision :',precision )
print ('Recall :',recall )
print ('F1 :',f1 )

In [ ]:
from sklearn.metrics import roc_curve, auc

# Assuming y_test and reconstruction_error have been defined
fpr, tpr, thresholds = roc_curve(y_test, reconstruction_error)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Extract the true labels and reconstruction errors from error_df
y_true = error_df['true_class']
y_scores = error_df['reconstruction_error']

# Compute the False Positive Rate (FPR) and True Positive Rate (TPR) for different thresholds
fpr, tpr, thresholds = roc_curve(y_true, y_scores)

# Compute the area under the ROC curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')

# Plot the diagonal line (random classifier)
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')

# Set plot labels and title
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")

# Display the plot
plt.show()


In [ ]:
print(f"ROC Score : {roc_auc}")

In [ ]:
# If error_df.true_class is a nested structure, flatten it
y_test = error_df.true_class.apply(lambda x: int(x[0][0]) if isinstance(x, list) else int(x))

# Ensure y_pred is a list of integers
y_pred = [0 if e < threshold else 1 for e in error_df.reconstruction_error.values]

# Now calculate the metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
print('Accuracy Score:', accuracy_score(y_test, y_pred))
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
